In [1]:
import os
import subprocess
import shutil
import wget
import glob

from collections import defaultdict
from Bio import SeqIO


os.chdir("/master/nplatt/pathogen_probes/")

major_groups = [ "anaplasma",         "apicomplexa",       "bacillus",         "bartonella",       
                 "borrelia",          "burkholderia",      "campylobacter",    "cestoda",         
                 "chlamydia",         "coxiella",          "ehrlichia",        "eurotiales",     
                 "francisella",       "hexamitidae",       "kinetoplastea",    "leptospira",
                 "listeria",          "mycobacterium",     "nematoda-clade1",  "nematoda-clade3",
                 "nematoda-clade4a",  "nematoda-clade4b",  "nematoda-clade5",  "onygenales",
                 "pasteurella",       "rickettsia",        "salmonella",       "streptobacillus",
                 "trematoda",         "tremellales",       "trypanosoma",      "yersinia" ]

# Design Probes

## Build probes via for each group via Phyluce

Run the phyluce pipleine that is embeded in taxon specific notebooks.

 - phyluce_probes_bact_ana-cam.ipynb
 - phyluce_probes_bact_chl-list.ipynb
 - phyluce_probes_bact_myc-yer.ipynb
 - phyluce_probes_euka.ipynb
 - phyluce_probes_euka-nematodes.ipynb
 
 ** THESE NOTEBOOKS ARE RUN WITH THE ```pathogen_probes-phyluce``` ENVIRONMENT **

## Identify target 50 loci and associated probes per group

In [26]:
#get 50 loci with the fewest numbers of probes
if os.path.exists("results/phyluce/targeted_probes"):
    shutil.rmtree("results/phyluce/targeted_probes")

os.makedirs("results/phyluce/targeted_probes")

for group in major_groups:
    in_clust_fas = "results/phyluce/{}/final_probe_design/{}_v1-master_probe_list.95P_cdhit".format( group, 
                                                                                                     group )
    
    #count number of probes per locus
    #read in the sequences
    seq_records = list(SeqIO.parse(in_clust_fas, "fasta"))
    
    locus_counts = defaultdict(int)
    locus_names = {}
    
    #loop through the sequences and count the number of probes per locus
    for seq in seq_records:
        locus=seq.name.split("_")[2]
        locus_counts[locus] = locus_counts[locus] + 1
    
    #get top 50 
    fifty_loci = sorted(locus_counts.items(), key=lambda x: x[1])[0:49]
    
    #count number of probes per group
    targeted_loci = []
    num_group_probes = 0
    
    for locus in fifty_loci:
        num_group_probes = num_group_probes + locus[1]
        targeted_loci.append(locus[0])
    
    print(group + " " + str(num_group_probes))
    
    #print probes to file
    targeted_probes = []
    for seq in seq_records:
        locus=seq.name.split("_")[2]
        if locus in targeted_loci:
            targeted_probes.append(seq)
            
    #write targeted_probes to file
    out_probe_file = "results/phyluce/targeted_probes/{}_50_loci_probes.fas".format(group)
    SeqIO.write(targeted_probes, out_probe_file, "fasta")

anaplasma 368
apicomplexa 3144
bacillus 834
bartonella 1799
borrelia 685
burkholderia 683
campylobacter 2198
cestoda 902
chlamydia 832
coxiella 144
ehrlichia 235
eurotiales 3766
francisella 471
hexamitidae 782
kinetoplastea 2914
leptospira 2522
listeria 767
mycobacterium 2464
nematoda-clade1 356
nematoda-clade3 1468
nematoda-clade4a 252
nematoda-clade4b 1596
nematoda-clade5 3138
onygenales 1892
pasteurella 616
rickettsia 395
salmonella 145
streptobacillus 245
trematoda 914
tremellales 1957
trypanosoma 616
yersinia 225


## make a test probe set

In [12]:
probe_file = "results/phyluce/50_loci_probes.fas"

!cat results/phyluce/targeted_probes/*_50_loci_probes.fas >results/phyluce/{probe_file}

## Replacing contamints from Arbor

I got a list of "contaminating" probes from Arbor and need to replace those.

In [8]:
if os.path.exists("results/phyluce/targeted_probes_post_contam_filtering"):
    shutil.rmtree("results/phyluce/targeted_probes_post_contam_filtering")

os.makedirs("results/phyluce/targeted_probes_post_contam_filtering")

In [9]:
#count the number of loci needed per group
with open('data/Probes_to_remove.txt', 'r',) as f:
    contam_probes = f.readlines()
    
contam_probes = [x.rstrip() for x in contam_probes]
# contam_probes = [x.rstrip() for x in contam_probes]

contam_loci = []
for probe in contam_probes:
    locus = "_".join(probe.split("_")[0:3])
    contam_loci.append(locus)
    
contam_loci = list(set(contam_loci))
contam_loci.sort()


In [10]:
# #get 50 loci with the fewest numbers of probes

for group in major_groups:
    in_clust_fas = "results/phyluce/{}/final_probe_design/{}_v1-master_probe_list.95P_cdhit".format( group, 
                                                                                                     group )
    #count number of probes per locus
    #read in the sequences
    seq_records = list(SeqIO.parse(in_clust_fas, "fasta"))
    
    locus_counts = defaultdict(int)
    locus_names = {}
    
    #loop through the sequences and count the number of probes per locus
    for seq in seq_records:
        locus="_".join(seq.name.split("_")[0:3])
        locus_counts[locus] = locus_counts[locus] + 1
    
    #get loci sorted by num probes 
    loci_sorted_by_n_probes = sorted(locus_counts.items(), key=lambda x: x[1])
    loci_sorted_by_n_probes = [x[0] for x in loci_sorted_by_n_probes]
    
    #remove contaminated loci
    for probe in contam_loci:
        if probe in loci_sorted_by_n_probes:
            loci_sorted_by_n_probes.remove(probe)

    #now get top 48
    target_loci = loci_sorted_by_n_probes[0:49]
    
    targeted_probes = []
    for seq in seq_records:
        locus="_".join(seq.name.split("_")[0:3])
        if locus in target_loci:
            targeted_probes.append(seq)
            
    #write targeted_probes to file
    out_probe_file = "results/phyluce/targeted_probes_post_contam_filtering/{}_48_loci_probes.fas".format(group)
    SeqIO.write(targeted_probes, out_probe_file, "fasta")

In [11]:
!cat results/phyluce/targeted_probes_post_contam_filtering/*_48_loci_probes.fas >results/phyluce/decon_probes.fas

# E-validation

In [2]:
if os.path.exists("results/peer_validation"):
    shutil.rmtree("results/peer_validation")

os.makedirs("results/peer_validation/sra_data")

all the seq data from PRJEB35488 was stored in a file ```results/phyluce/peer_validation/sra_data/SraAccList.txt``` and used to download the SRA files.

In [3]:
peromyscus_samples = { "peer099" : "ERR3672161",   "peer098" : "ERR3672162",
                       "peer095" : "ERR3672163",   "peer093" : "ERR3672164",
                       "peer092" : "ERR3672165",   "peer091" : "ERR3672166",
                       "peer075" : "ERR3672167",   "peer339" : "ERR3672168",
                       "peer338" : "ERR3672169",   "peer336" : "ERR3672170",
                       "peer335" : "ERR3672171",   "peer334" : "ERR3672172",
                       "peer333" : "ERR3672173",   "peer330" : "ERR3672174",
                       "peer326" : "ERR3672175",   "peer305" : "ERR3672176",
                       "peer017" : "ERR3672177",   "peer016" : "ERR3672178",
                       "peer015" : "ERR3672179",   "peer144" : "ERR3672180",
                       "peer139" : "ERR3672181",   "peer138" : "ERR3672182",
                       "peer136" : "ERR3672183",   "peer135" : "ERR3672184",
                       "peer134" : "ERR3672185",   "peer012" : "ERR3672186"  }

## Download the data

This takes a LONG LONG time (~1 day).

In [6]:
!prefetch --option-file data/SraAccList.txt


2020-02-20T17:09:42 prefetch.2.8.0: 1) 'ERR3672161' is found locally

2020-02-20T17:09:42 prefetch.2.8.0: 2) 'ERR3672162' is found locally

2020-02-20T17:09:43 prefetch.2.8.0: 3) 'ERR3672163' is found locally

2020-02-20T17:09:43 prefetch.2.8.0: 4) 'ERR3672164' is found locally

2020-02-20T17:09:44 prefetch.2.8.0: 5) 'ERR3672165' is found locally

2020-02-20T17:09:44 prefetch.2.8.0: 6) 'ERR3672166' is found locally

2020-02-20T17:09:45 prefetch.2.8.0: 7) 'ERR3672167' is found locally

2020-02-20T17:09:45 prefetch.2.8.0: 8) 'ERR3672168' is found locally

2020-02-20T17:09:45 prefetch.2.8.0: 9) 'ERR3672169' is found locally

2020-02-20T17:09:46 prefetch.2.8.0: 10) 'ERR3672170' is found locally

2020-02-20T17:09:46 prefetch.2.8.0: 11) 'ERR3672171' is found locally

2020-02-20T17:09:47 prefetch.2.8.0: 12) 'ERR3672172' is found locally

2020-02-20T17:09:47 prefetch.2.8.0: 13) 'ERR3672173' is found locally

2020-02-20T17:09:48 prefetch.2.8.0: 14) 'ERR3672174' is found locally

2020-02-20T17:

## Convert to fas and randomize

In [9]:
for sample in peromyscus_samples:
    
    #for each sample, cover the sra a to fasta
    accession = peromyscus_samples[sample]
    dump_cmd = "fastq-dump --split-spot --outdir results/peer_validation/sra_data/ --split-files --fasta 0 {}; ".format(accession)
    subprocess.run(cmd.split(" "))
    
    #combine into a single fasta
    r1_fas   = "results/peer_validation/sra_data/" + accession + "_1.fasta"
    r2_fas   = "results/peer_validation/sra_data/" + accession + "_2.fasta"
    comb_fas = "results/peer_validation/sra_data/" + accession + ".fasta"

    #combine
    cat_cmd = "cat {} {} >{}; ".format(r1_fas, r2_fas, comb_fas)
    
    #delete single end reads
    rm_cmd = "rm {} {}; ".format(r1_fas, r2_fas)
    
    #then take the fasta (raw reads) and randomize the nucleotide order per read
    #  this creates a control, but maintains the GC% with muliple ranomizing passes per read
    in_fasta  = "results/peer_validation/sra_data/" + accession + ".fasta"
    out_fasta = "results/peer_validation/sra_data/" + accession + "_randomized.fasta"
    n_passes  = 5 
    
    random_cmd = "python code/randomize_nucleotide_order.py {} {} {}; ".format(n_passes, in_fasta, out_fasta)
    
    #now set up qsub
    jid= "random_" + accession
    log= "results/peer_validation/sra_data/" + jid + ".log"
    qsub_cmd = "qsub -V -cwd -S /bin/bash -q all.q -j y -N {} -o {} -pe smp 12".format(jid, log)

    #conda
    conda_cmd = "conda activate pathogen_probes; "

    #and submit to the queue
    combined_cmd ="echo \"" + conda_cmd + dump_cmd + cat_cmd + rm_cmd + random_cmd + "\" | " + qsub_cmd
    
    #print(combined_cmd)
    !{combined_cmd}
    

Your job 5489754 ("random_ERR3672161") has been submitted
Your job 5489755 ("random_ERR3672162") has been submitted
Your job 5489756 ("random_ERR3672163") has been submitted
Your job 5489757 ("random_ERR3672164") has been submitted
Your job 5489758 ("random_ERR3672165") has been submitted
Your job 5489759 ("random_ERR3672166") has been submitted
Your job 5489760 ("random_ERR3672167") has been submitted
Your job 5489761 ("random_ERR3672168") has been submitted
Your job 5489762 ("random_ERR3672169") has been submitted
Your job 5489763 ("random_ERR3672170") has been submitted
Your job 5489764 ("random_ERR3672171") has been submitted
Your job 5489765 ("random_ERR3672172") has been submitted
Your job 5489766 ("random_ERR3672173") has been submitted
Your job 5489767 ("random_ERR3672174") has been submitted
Your job 5489768 ("random_ERR3672175") has been submitted
Your job 5489769 ("random_ERR3672176") has been submitted
Your job 5489770 ("random_ERR3672177") has been submitted
Your job 54897

## Build indexes and blast

In [10]:
if os.path.exists("results/peer_validation/probe_blasts"):
    shutil.rmtree("results/peer_validation/probe_blasts")

os.makedirs("results/peer_validation/probe_blasts")

In [11]:
#get the P. californicus genome
ncbi_link = "ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/007/827/085/GCA_007827085.1_PCal_1.0/GCA_007827085.1_PCal_1.0_genomic.fna.gz"

filename = wget.download(ncbi_link)
os.rename(filename, "results/peer_validation/probe_blasts/Pcal1.fas.gz")

#mv filename to appropriate directory
!gunzip results/peer_validation/probe_blasts/Pcal1.fas.gz

In [13]:
#make probe set with pero genome
probes_plus_genome_fas = "results/peer_validation/probe_blasts/probes_and_Pcal1.fas"

#combine it with the probes to
!cat results/peer_validation/probe_blasts/Pcal1.fas {probe_file} >{probes_plus_genome_fas}

In [ ]:

#index probes and genome
subject_fas = probes_plus_genome_fas
fas_db_cmd  = "makeblastdb -dbtype nucl -in {}".format(subject_fas)

subprocess.run(fas_db_cmd.split(" "))


for sample in peromyscus_samples:
    accession = peromyscus_samples[sample]
    
    ##########################################################################################
    in_fas     = "results/peer_validation/sra_data/" + accession + ".fasta"
    out_blast  = "results/peer_validation/probe_blasts/" + accession + "_blast.out"
    blast_cmd = "blastn -db {} -query {} -out {} -perc_identity 90 -outfmt 6 -num_threads 6".format(subject_fas, 
                                                                                                    in_fas, 
                                                                                                    out_blast)
    
    #now set up qsub
    jid= "blast_" + accession
    log= "results/peer_validation/probe_blasts/" + jid + ".log"
    qsub_cmd = "qsub -V -cwd -S /bin/bash -q all.q -j y -N {} -o {} -pe smp 12".format(jid, log)

    #conda
    conda_cmd = "conda activate pathogen_probes; "

    #and submit to the queue
    combined_cmd ="echo \"" + conda_cmd + blast_cmd + "\" | " + qsub_cmd

    #print(combined_cmd)
    !{combined_cmd}
    
    ##########################################################################################
    rand_fas   = "results/peer_validation/sra_data/" + accession + "_randomized.fasta"
    rand_blast = "results/peer_validation/probe_blasts/" + accession + "_randomized_blast.out"
    rand_cmd  = "blastn -db {} -query {} -out {} -perc_identity 90 -outfmt 6 -num_threads 6".format(subject_fas, 
                                                                                                    rand_fas, 
                                                                                                    rand_blast)
    #now set up qsub
    jid= "blast_random_" + accession
    log= "results/peer_validation/probe_blasts/" + jid + ".log"
    qsub_cmd = "qsub -V -cwd -S /bin/bash -q all.q -j y -N {} -o {} -pe smp 12".format(jid, log)

    #conda
    conda_cmd = "conda activate pathogen_probes; "

    #and submit to the queue
    combined_cmd ="echo \"" + conda_cmd + blast_cmd + "\" | " + qsub_cmd

    #print(combined_cmd)
    !{combined_cmd}

In [ ]:
for sample in peromyscus_samples:
    accession = peromyscus_samples[sample]
    
    fasta      = "results/peer_validation/sra_data/" + accession + ".fasta"
    rand_fasta = "results/peer_validation/sra_data/" + accession + "_randomized.fasta"

    
    #submit to build blast index of real and random data
    fas_db_cmd  = "makeblastdb -dbtype nucl -in {}".format(fasta)
    rand_db_cmd = "makeblastdb -dbtype nucl -in {}".format(rand_fasta)
    
    #now set up qsub
    jid= "db_" + accession
    log= "results/peer_validation/sra_data/" + jid + ".log"
    qsub_cmd = "qsub -V -cwd -S /bin/bash -q all.q -j y -N {} -o {} -pe smp 12".format(jid, log)

    #conda
    conda_cmd = "conda activate pathogen_probes; "

    #and submit to the queue
    combined_cmd ="echo \"" + conda_cmd + random_cmd + "\" | " + qsub_cmd
    
    #print(combined_cmd)
    !{combined_cmd}

## Kraken2 the unfiltered reads

In [ ]:
#test probes against several SRAs
if os.path.exists("results/peer_validation/kraken2"):
    shutil.rmtree("results/peer_validation/kraken2")

os.makedirs("results/peer_validation/kraken2")

In [ ]:
#build Kraken2 db
subprocess.rub("kraken2-build --standard --threads 6 --db results/peer_validation/kraken2_standard_db", shell=True))

In [ ]:
    #quanify/classify
    kraken2 \
        --use-names \
        --threads 4 \
        --db kraken2_standard_db \
        --report kraken.out \
        reads.fa \
        >kraken.tbl

## Do something to compare